<img src="https://industrial.uniandes.edu.co/sites/default/files/imagenes/uniandeslogo.png" alt="Universidad de los Andes" style="float: right; width: 300px; height: auto;">

# Cleaning COVID

Autor: Juan Diego Heredia Niño 

Email: jd.heredian@uniandes.edu.co

Date: Jan 2026

In [1]:
# Import necessary libraries
import pandas as pd  # For data manipulation and analysis
import yaml  # To read YAML configuration files
from pathlib import Path  # For cross-platform file path handling

In [2]:
# Load directory paths from configuration file
with open('paths.yml', 'r') as file:
    paths = yaml.safe_load(file)  # Read and parse YAML file

# Create Path objects for each directory
raw = Path(paths['data']['raw'])  # Directory with raw data
temp = Path(paths['data']['temp'])  # Directory with temporary processed data
processed = Path(paths['data']['processed'])  # Directory with final processed data

In [3]:
df_casos = pd.read_csv(raw/'minsalud'/'Casos_positivos_de_COVID-19_en_Colombia._20260215.csv', low_memory=False, encoding='utf-8')

In [4]:
df_casos['Fecha de inicio de síntomas'] = pd.to_datetime(df_casos['Fecha de inicio de síntomas'])

df_casos['mun_code'] = df_casos['Código DIVIPOLA municipio'].str.replace(r'[^\d]', '', regex=True).astype(float).astype(int).astype(str).str.zfill(5)

df_casos['quarter'] = df_casos['Fecha de inicio de síntomas'].dt.to_period('Q')

df_casos_por_semestre = df_casos.groupby(['quarter','mun_code']).size().reset_index(name='covid')

df_casos_por_semestre.to_parquet(temp/'minsalud'/'casos_covid_por_semestre.parquet', index=False)

In [5]:
df_muertes = pd.read_csv(raw/'minsalud'/'Fallecidos_COVID_en_Colombia_20260215.csv', low_memory=False, encoding='utf-8')

In [6]:
df_muertes['Fecha de inicio de síntomas'] = pd.to_datetime(df_muertes['Fecha de inicio de síntomas'])

df_muertes['mun_code'] = df_muertes['Código DIVIPOLA municipio'].str.replace(r'[^\d]', '', regex=True).astype(float).astype(int).astype(str).str.zfill(5)

df_muertes['quarter'] = df_muertes['Fecha de inicio de síntomas'].dt.to_period('Q')

df_muertes_por_semestre = df_muertes.groupby(['quarter','mun_code']).size().reset_index(name='covid_d')

df_muertes_por_semestre.to_parquet(temp/'minsalud'/'muertes_covid_por_semestre.parquet', index=False)

Podemos hacer la suma agregada por municipio para ambos y sacar un ratio de poblacion contagiada en ese momento, contagiada acumulada, esto dos relativos a la poblacion total y para las muertes relativo a contagiados y poblacion total